In [16]:
import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
import cPickle
import random
import math
import h5py
from multiprocessing import cpu_count
import scipy.io as sio
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict
from gensim.models import Word2Vec, KeyedVectors
from six import iteritems
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle as skshuffle
from numpy import genfromtxt

from collections import defaultdict as dd
import tensorflow as tf
from tensorflow.python.ops.init_ops import glorot_uniform_initializer, RandomUniform, RandomNormal

from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec
from gensim.models.word2vec import Vocab

np.random.seed(1)
random.seed(1)

In [18]:
# Initialize parameters
DATASET = 'blogcatalog'

embedding_size = 128
learning_rate = 0.1
gl_learning_rate = 0.1
batch_size = 200
neg_samp = 0
model_file = 'trans.model'

window_size = 10
path_size = 10


use_feature = True
update_emb = True
layer_loss =  True

In [78]:
file_list = ['/hdd2/graph_embedding/customized/blogcatalog.embeddings.walks.0']
dataset = genfromtxt(file_list[0], delimiter=' ')

def get_num_vacabulary(dataset):
    word_count = 0
    for d in dataset:
        word_count = max(word_count, max(d))
    return int(word_count)

vocabulary_size = get_num_vacabulary(dataset) + 1

In [55]:
words = dataset.flatten()
words = [str(w) for w in data]

In [80]:
def build_dataset(words):
    count = []
#     count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
        data.append(index)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

('Most common words (+UNK)', [('1', 196735), ('2', 195187), ('3', 169490), ('4', 147968), ('5', 137221)])
('Sample data', [1193, 2876, 225, 457, 19, 34, 5665, 37, 3046, 29])


In [81]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
#             print(batch[i * num_skips + j])
#             print(buffer[skip_window])
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

# print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(1, 10)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])


with num_skips = 1 and skip_window = 10:
('    batch:', ['1978', '803', '5884', '1526', '33', '884', '169', '1851'])
('    labels:', ['1851', '30', '20', '3048', '30', '195', '782', '884'])


In [82]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 10 # How many words to consider left and right.
num_skips = 1 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Variables.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                                   labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)

    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [83]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    batch_data, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l, res_embed = session.run([optimizer, loss, embed], feed_dict=feed_dict)
    print('loss: %.4f ' %l)

loss: 5.5755 


In [84]:
num_steps = 100001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
#         if step % 10000 == 0:
#             sim = similarity.eval()
#             for i in range(valid_size):
#                 valid_word = reverse_dictionary[valid_examples[i]]
#                 top_k = 8 # number of nearest neighbors
#                 nearest = (-sim[i, :]).argsort()[1:top_k+1]
#                 log = 'Nearest to %s:' % valid_word
#                 for k in range(top_k):
#                     close_word = reverse_dictionary[nearest[k]]
#                     log = '%s %s,' % (log, close_word)
#                 print(log)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 5.315740
Average loss at step 2000: 4.626085
Average loss at step 4000: 4.395991
Average loss at step 6000: 4.342272
Average loss at step 8000: 4.314121
Average loss at step 10000: 4.293211
Average loss at step 12000: 4.278658
Average loss at step 14000: 4.262972
Average loss at step 16000: 4.256954
Average loss at step 18000: 4.249532
Average loss at step 20000: 4.241829
Average loss at step 22000: 4.230031
Average loss at step 24000: 4.225935
Average loss at step 26000: 4.221833
Average loss at step 28000: 4.218340
Average loss at step 30000: 4.214572
Average loss at step 32000: 4.210010
Average loss at step 34000: 4.206765
Average loss at step 36000: 4.200252
Average loss at step 38000: 4.199512
Average loss at step 40000: 4.194152
Average loss at step 42000: 4.192305
Average loss at step 44000: 4.188063
Average loss at step 46000: 4.184381
Average loss at step 48000: 4.186216
Average loss at step 50000: 4.180659
Average loss at step 52000: 4.1788

In [86]:
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.txt'
np.savetxt(embedding_filename, final_embeddings)

In [85]:
embedding_filename

NameError: name 'embedding_filename' is not defined

In [5]:
def initialize_parameters(l_emd_f_W_size, l_x_hid_W_size, l_y_W_size):
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [4, 4, 3, 8]??
                        W2 : [2, 2, 8, 16]?
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)
    
    l_emd_f_W = tf.Variable(tf.truncated_normal(l_emd_f_W_size,
                            stddev=1.0 / math.sqrt(l_emd_f_W_size[1])))
    l_emd_f_b = tf.Variable(tf.zeros([l_emd_f_W_size[0]]))
    
    l_x_hid_W = tf.get_variable('l_x_hid_W', shape = l_x_hid_W_size,
                               initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_x_hid_b = tf.get_variable('l_x_hid_b', shape = [l_x_hid_W_size[0], 1],
                               initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_y_W = tf.get_variable('l_y_W', shape = l_y_W_size,
                           initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_y_b = tf.get_variable('l_y_b', shape = [l_y_W_size[0], 1],
                           initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    
    parameters = {'l_emd_f_W': l_emd_f_W,
                  'l_emd_f_b': l_emd_f_b,
                  'l_x_hid_W': l_x_hid_W,
                  'l_x_hid_b': l_x_hid_b,
                  'l_y_W': l_y_W,
                  'l_y_b': l_y_b}
    
    return parameters

In [9]:
def create_placeholders():
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    
    x_sym = tf.placeholder(tf.float32, shape = [None, x.shape[1]], name = 'x')
    y_sym = tf.placeholder(tf.int32, shape = [None, y.shape[1]], name = 'y')
    g_sym = tf.placeholder(tf.int32, shape = [None, ], name = 'g')
    gy_sym = tf.placeholder(tf.int32, shape = [None, 1], name = 'gy')
    ind_sym = tf.placeholder(tf.int32, shape = [None], name = 'ind')
    
    path_sym = tf.placeholder(tf.int32, shape = [batch_size, path_size+1], name = 'path')
    path_id_sym = tf.placeholder(tf.int32, shape = [batch_size, ], name = 'path_id')
    w_path2pair_sym = tf.placeholder(tf.float32, shape = [batch_size, None], name = 'w_path2pair_sym')
    
    
    return x_sym, y_sym, g_sym, gy_sym, ind_sym, path_sym, path_id_sym, w_path2pair_sym

In [10]:
file_list = ['/hdd2/graph_embedding/customized/blogcatalog.embeddings.walks.0']
dataset = genfromtxt(file_list[0], delimiter=' ')

def get_num_vacabulary(dataset):
    count = 0
    for d in dataset:
        count = max(count, max(d))
    return int(count)

In [11]:
# num_sampled = 64
# num_ver = max(graph.keys()) + 1
# vocabulary_size = num_ver
vocabulary_size = get_num_vacabulary(dataset) + 1
n_hidden = 32
n_steps = path_size+1 #path length
n_input = 128 # embedding dim

In [12]:
vocabulary_size

10312

In [17]:
# def build():
"""
Builds the model.
"""
np.random.seed(1)
random.seed(1)
tf.random_normal_initializer(seed = 1)

cgraph = tf.Graph()

with cgraph.as_default(), tf.device('/cpu:0'):

    x_sym, y_sym, g_sym, gy_sym, ind_sym, path_sym, path_id_sym, w_path2pair_sym = create_placeholders()

    # word embedding
    tf.random_normal_initializer(seed = 1)
    tf.set_random_seed(1)
    
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
#     softmax_weights = tf.Variable(
#         tf.random_normal([vocabulary_size, embedding_size], mean = 0.0, stddev = 0.01))
    softmax_weights = tf.Variable(
          tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
   
    l_emd_f = tf.nn.embedding_lookup(embeddings, g_sym)

    g_loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases,
                                   inputs = l_emd_f, labels = gy_sym, 
                                   num_sampled = 5, 
                                   num_classes = vocabulary_size))    
    g_optimizer = tf.train.AdamOptimizer(0.025).minimize(g_loss)
    
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm

In [18]:
def count_textfiles(file_list):
    count = -1
    num_path = 0
    for file_name in file_list:
        with open(file_name, 'r') as f:
            for line in f:
                num_path += 1
                for l in line.split():
                    count = max(count, int(l))
    return count, num_path

def get_word_pair(line):
    g = []
    list_path = []
    path = [int(t) for t in line.split()]
    list_path.append(path)
    for l in range(len(path)):
        for m in range(l - window_size, l + window_size + 1):
            if m < 0 or m >= len(path) or m == l: continue
            g.append([path[l], path[m]])
            gy.append(1.0)
    return g, list_path

def gen_graph_from_path_collection(data):
    g = []
    list_path = []
    
    for line in data:
        for l in xrange(len(line)):
            for m in xrange(1 - window_size, l + window_size + 1):
                if (m < 0 or m >= len(line) or m == l): 
                    continue
                g.append([int(line[l]), int(line[m])])
    return np.array(g, dtype = np.int32)

In [19]:
window_size = 10
representation_size = 128
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.txt'

In [ ]:
# unsupervised word2vec
init_iter_deepwalk = 10
config = tf.ConfigProto(allow_soft_placement=True)
tot = 0
batch_path = 10000
with tf.Session(graph = cgraph, config=config) as session:
    tf.global_variables_initializer().run()
    

    for i in xrange(init_iter_deepwalk):
        for batch_id in xrange(len(dataset) / batch_path + 1):
            start_idx = batch_path * batch_id
            end_idx = min(len(dataset), batch_path * (batch_id + 1))
            data_batch = dataset[start_idx: end_idx, :]
            gx = gen_graph_from_path_collection(data_batch)
            feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
            _, l  = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
            
            print('iter: %d/%d, loss: %.4f' %(batch_id, len(dataset) / batch_path + 1, l))
    #             feed_dict={g_sym: np.array(range(0,v+1)), gy_sym: np.array(range(0,v+1)).reshape(v+1,1)}
    #             res_l_emd_f = session.run(l_emd_f, feed_dict=feed_dict)
#                 final_embeddings = normalized_embeddings.eval()
#                 embedding_filename_tmp = '/hdd2/graph_embedding/customized/blog_embeddings.%d.txt' %(tot)
    #             np.savetxt(embedding_filename_tmp, res_l_emd_f)
        tmp_embeddings = normalized_embeddings.eval()
        embedding_filename_tmp = '/hdd2/graph_embedding/customized/blog_embeddings.iter%d.txt' %(i)
        np.savetxt(embedding_filename_tmp, tmp_embeddings)
        
    
    final_embeddings = normalized_embeddings.eval()

In [87]:
np.savetxt(embedding_filename, final_embeddings)

In [96]:
l

1.0539942

iter: 0/83, loss: 1.0540


In [7]:
class Skipgram(Word2Vec):
    """A subclass to allow more customization of the Word2Vec internals."""
    
    def __init__(self, vocabulary_counts=None, **kwargs):

        self.vocabulary_counts = None

        kwargs["min_count"] = kwargs.get("min_count", 0)
        kwargs["workers"] = kwargs.get("workers", cpu_count())
        kwargs["size"] = kwargs.get("size", 128)
        kwargs["sentences"] = kwargs.get("sentences", None)
        kwargs["window"] = kwargs.get("window", 10)
        kwargs["sg"] = 1
        kwargs["hs"] = 1

        if vocabulary_counts != None:
            self.vocabulary_counts = vocabulary_counts

        super(Skipgram, self).__init__(**kwargs)
        
class WalksCorpus(object):
    def __init__(self, file_list):
        self.file_list = file_list
    def __iter__(self):
        for file in self.file_list:
            with open(file, 'r') as f:
                for line in f:
                    yield line.split()

In [26]:
import itertools
walks_corpus = WalksCorpus(file_list)

In [36]:
i = 0
for line in walks_corpus:
    i += 1
print(i)

824960


In [21]:
# model = Skipgram(sentences=walks_corpus, vocabulary_counts=vocabulary_size, size=128,
#                  window=10, min_count=0, trim_rule=None, workers=1)

In [31]:
1

1

In [6]:
def sparse2graph(x):
    G = defaultdict(lambda: set())
    cx = x.tocoo()
    for i,j,v in zip(cx.row, cx.col, cx.data):
        G[i].add(j)
    return {str(k): [str(x) for x in v] for k,v in iteritems(G)}

In [7]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [55]:
def scoring(emb_filename, matfile):
    # 0. Files
    embeddings_file = emb_filename

    # 1. Load Embeddings
#     embeddings = np.loadtxt(embeddings_file)
    
    ## for original deepwalk
    #model = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)
    
    ## for external word2vec lib
    model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')

    # 2. Load labels
    mat = sio.loadmat(matfile)
    A = mat['network']
    graph = sparse2graph(A)
    labels_matrix = mat['group']
    labels_count = labels_matrix.shape[1]
    mlb = MultiLabelBinarizer(range(labels_count))

    # Map nodes to their features (note:  assumes nodes are labeled as integers 1:N)
#     features_matrix = embeddings

#     features_matrix = np.asarray([model[str(node)] for node in range(len(graph))])

    # use other word2vec lib
    features_matrix = np.asarray([model[str(node)] for node in xrange(len(model.vocab)-1)])
    
    # 2. Shuffle, to create train/test groups
    shuffles = []
    for x in range(1):
        shuffles.append(skshuffle(features_matrix, labels_matrix))

    # 3. to score each train/test group
    all_results = defaultdict(list)

#     if args.all:
#         training_percents = numpy.asarray(range(1, 10)) * .1
#     else:
#         training_percents = [0.1, 0.5, 0.9]
    training_percents = [0.1]
    for train_percent in training_percents:
        for shuf in shuffles:
            
            X, y = shuf

            training_size = int(train_percent * X.shape[0])

            X_train = X[:training_size, :]
            y_train_ = y[:training_size]

            y_train = [[] for x in range(y_train_.shape[0])]


            cy =  y_train_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_train[i].append(j)

            assert sum(len(l) for l in y_train) == y_train_.nnz

            X_test = X[training_size:, :]
            y_test_ = y[training_size:]

            y_test = [[] for _ in range(y_test_.shape[0])]

            cy =  y_test_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_test[i].append(j)

            clf = TopKRanker(LogisticRegression())
            clf.fit(X_train, y_train_)

            # find out how many labels should be predicted
            top_k_list = [len(l) for l in y_test]
            preds = clf.predict(X_test, top_k_list)

            results = {}
            averages = ["micro", "macro"]
            for average in averages:
                results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)

            all_results[train_percent].append(results)

    print ('Results, using embeddings of dimensionality', X.shape[1])
    print ('-------------------')
    for train_percent in sorted(all_results.keys()):
        print ('Train percent:', train_percent)
    for index, result in enumerate(all_results[train_percent]):
        print ('Shuffle #%d:   ' % (index + 1), result)
    avg_score = defaultdict(float)
    for score_dict in all_results[train_percent]:
        for metric, score in iteritems(score_dict):
            avg_score[metric] += score
    for metric in avg_score:
        avg_score[metric] /= len(all_results[train_percent])
    print ('Average score:', dict(avg_score))
    print ('-------------------')

In [58]:
matfile = '/hdd2/graph_embedding/deepwalk/example_graphs/blogcatalog.mat'
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.iter2.txt'
embedding_filename_original = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
embedding_filename_other_lib = '/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin'

scoring('', matfile)

('Results, using embeddings of dimensionality', 128)
-------------------
('Train percent:', 0.1)
('Shuffle #1:   ', {'micro': 0.35389710412815772, 'macro': 0.16275639403364192})
('Average score:', {'micro': 0.35389710412815772, 'macro': 0.16275639403364192})
-------------------


/home/mingzeng/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 37 is present in all training examples.
  str(classes[c]))


In [23]:
embedding_filename_original = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
model = KeyedVectors.load_word2vec_format(embedding_filename_original, binary=False)
                                          
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.iter2.txt'
embeddings = np.loadtxt(embedding_filename)


In [22]:
model['0']

array([ 0.110529  ,  0.19354101,  0.3969    ,  0.16305301, -0.198276  ,
       -0.219273  ,  0.045498  ,  0.138824  , -0.174495  ,  0.20618799,
        0.48984301,  0.302856  , -0.46013999,  0.227097  ,  0.071323  ,
       -0.43010399,  0.015921  , -0.021136  ,  0.28175899, -0.26225701,
        0.102745  , -0.31963301,  0.058964  ,  0.132149  ,  0.176946  ,
       -0.155433  ,  0.160973  , -0.282166  ,  0.030017  , -0.165079  ,
        0.25168899, -0.52189702,  0.005448  ,  0.18592501, -0.013992  ,
       -0.070675  ,  0.033961  ,  0.117675  , -0.073083  ,  0.068748  ,
       -0.10755   ,  0.068676  ,  0.17162   , -0.136898  ,  0.17979699,
       -0.106551  , -0.212037  ,  0.103523  , -0.242975  , -0.46731299,
        0.182107  ,  0.092075  , -0.141946  ,  0.051342  ,  0.31653801,
        0.28085399, -0.029812  ,  0.19881   ,  0.31846499, -0.12293   ,
        0.22415499, -0.21315201, -0.220193  , -0.15813901,  0.104372  ,
       -0.038969  ,  0.020534  , -0.18385699, -0.049377  ,  0.24

In [26]:
embeddings[0]

array([ -1.47322476e-01,   2.67052501e-02,   1.52672976e-02,
        -1.37635857e-01,  -6.47586957e-02,  -2.46349778e-02,
        -8.18009302e-02,   6.91710338e-02,  -9.66271833e-02,
        -9.89247933e-02,   5.70955575e-02,   1.53366596e-01,
         8.30798447e-02,  -1.44452751e-01,   9.11711424e-04,
         1.01996966e-01,   4.97803837e-02,  -4.91250493e-02,
         1.03872299e-01,  -1.11238003e-01,   3.35200056e-02,
         6.37169033e-02,  -3.92291024e-02,  -2.92820130e-02,
        -2.85616820e-03,  -3.53677608e-02,  -5.77375665e-02,
        -3.55093554e-02,   4.43368331e-02,  -1.63600937e-01,
        -5.06901741e-02,   1.09737791e-01,  -6.03695633e-03,
        -9.09792027e-04,  -9.44678709e-02,  -5.35581484e-02,
         5.67527264e-02,  -2.60556079e-02,   1.89456977e-02,
         2.38609854e-02,   4.84210551e-02,   1.34210438e-01,
        -1.30895942e-01,   6.51589632e-02,   6.92558140e-02,
        -5.88307390e-03,  -1.87402926e-02,   1.52439341e-01,
        -7.65966475e-02,

In [7]:
mat = sio.loadmat(matfile)
A = mat['network']
labels_matrix = mat['group']
labels_count = labels_matrix.shape[1]
mlb = MultiLabelBinarizer(range(labels_count))

TypeError: 'MultiLabelBinarizer' object is not iterable

In [27]:
# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [28]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2018-03-13 21:07:11,853 : INFO : collecting all words and their counts
2018-03-13 21:07:11,855 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-13 21:07:11,857 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2018-03-13 21:07:11,858 : INFO : Loading a fresh vocabulary
2018-03-13 21:07:11,860 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-03-13 21:07:11,863 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-03-13 21:07:11,865 : INFO : deleting the raw counts dictionary of 3 items
2018-03-13 21:07:11,868 : INFO : sample=0.001 downsamples 3 most-common words
2018-03-13 21:07:11,870 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-03-13 21:07:11,872 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2018-03-13 21:07:11,874 : INFO : resetting layer weights
2018-03-13 21:07:11,876 : INFO : training model with 3 workers o

In [87]:
init_iter_label = 10
init_iter_graph = 7
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

In [17]:
max_iter = 1
iter_graph = 0
iter_inst = 5
iter_label = 0
use_reweight = False

In [92]:
# with tf.Session() as session:
#     gx, gy = next(label_generator)
#     tf.global_variables_initializer().run()
#     feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#     _, loss = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
#     print 'iter label', i, loss

In [93]:
res_reweight[0]

array([ 0.00497301], dtype=float32)

In [353]:
# with tf.Session() as session:
# #     tf.global_variables_initializer().run()
#     feed_dict={x_sym: xs, y_sym: ys}
#     ll, l = session.run([optimizer, loss], feed_dict=feed_dict)

In [14]:
import word2vec

In [15]:
word2vec.word2phrase('/hdd2/graph_embedding/tmp/text8', '/hdd2/graph_embedding/tmp/text8-phrases', verbose=True)

Starting training using file /hdd2/graph_embedding/tmp/text8
Words processed: 17000K     Vocab size: 4399K  
Vocab size (unigrams + bigrams): 2419827
Words in train file: 17005206


In [16]:
word2vec.word2vec('/hdd2/graph_embedding/tmp/text8-phrases', '/hdd2/graph_embedding/tmp/text8.bin', size=100, verbose=True)

Starting training using file /hdd2/graph_embedding/tmp/text8-phrases
Vocab size: 98331
Words in train file: 15857306
Alpha: 0.000002  Progress: 100.04%  Words/thread/sec: 467.25k  

In [20]:
word2vec.word2vec('/hdd2/graph_embedding/customized/blogcatalog.embeddings.walks.0',
                  '/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin', 
                  size=128, verbose=True)

Starting training using file /hdd2/graph_embedding/customized/blogcatalog.embeddings.walks.0
Vocab size: 10313
Words in train file: 33823360
Alpha: 0.000002  Progress: 100.00%  Words/thread/sec: 409.50k  

In [22]:
model = word2vec.load('/hdd2/graph_embedding/tmp/blogcatalog.embeddings.walks.0.bin')

In [40]:
model.vocab

array([u'</s>', u'4838', u'175', ..., u'9756', u'1678', u'2931'], 
      dtype='<U78')

In [52]:
features_matrix = np.asarray([model[str(node)] for node in xrange(len(model.vocab)-1)])

In [48]:
model.vocab

array([u'</s>', u'4838', u'175', ..., u'9756', u'1678', u'2931'], 
      dtype='<U78')

In [38]:
model.vectors[1]

array([ 0.03300432, -0.07565996, -0.16463131, -0.10145742,  0.00168844,
       -0.09007662, -0.03259023, -0.22714335, -0.00282227, -0.05083675,
       -0.07032426, -0.15832779, -0.13998848,  0.07574239,  0.01739285,
        0.05170207, -0.01704509, -0.04309488,  0.13984987, -0.07098495,
       -0.08131668, -0.06108802,  0.03238546,  0.03199002, -0.14047897,
        0.1192169 , -0.11983237,  0.17141342, -0.04238614, -0.07682881,
       -0.09226497, -0.08641661, -0.05119943,  0.04880232,  0.05202007,
        0.11911941,  0.02494194,  0.0460702 , -0.02544389,  0.02227429,
       -0.02918422,  0.08963519, -0.01449389,  0.13147458,  0.04316   ,
        0.05062373,  0.01030556,  0.14064927,  0.05300813,  0.09054667,
        0.16458763,  0.00496632,  0.09853401, -0.09157445,  0.03466847,
       -0.192109  , -0.04286476,  0.0337339 ,  0.09414463,  0.02883872,
       -0.12935385,  0.05580147,  0.04444493,  0.12305645, -0.06843153,
       -0.05650309, -0.06930542, -0.0933516 , -0.08945961,  0.03

In [50]:
model['0']

array([-0.03359349, -0.01812873, -0.00173358,  0.01964512,  0.10055595,
        0.02292307,  0.0607472 ,  0.04138128, -0.12003173, -0.09744611,
        0.07093195, -0.01193107,  0.07130294,  0.08676811,  0.11344255,
        0.05344105, -0.02284033,  0.05260049, -0.01932774, -0.09711266,
       -0.19951601,  0.00738851, -0.07712144, -0.07025649, -0.03221126,
       -0.13667656,  0.08364721, -0.04125704, -0.03211425,  0.06983157,
       -0.00799658, -0.01177289, -0.16706869,  0.07463736, -0.09091619,
       -0.11201481,  0.08337475,  0.05930549, -0.07940312, -0.05631904,
       -0.01991395, -0.02349721,  0.1824768 ,  0.01132163, -0.04610476,
        0.01006832,  0.04507252, -0.08544502,  0.02825145,  0.02216966,
        0.03565323, -0.06984831,  0.03623525,  0.1129318 , -0.09319748,
       -0.04915803,  0.11094059,  0.0029751 , -0.08399934,  0.21738739,
        0.06743176, -0.09108514,  0.01798326, -0.08503798,  0.0290214 ,
        0.07596923, -0.02369601,  0.11986063, -0.04430954, -0.07

In [53]:
features_matrix[0]

array([-0.03359349, -0.01812873, -0.00173358,  0.01964512,  0.10055595,
        0.02292307,  0.0607472 ,  0.04138128, -0.12003173, -0.09744611,
        0.07093195, -0.01193107,  0.07130294,  0.08676811,  0.11344255,
        0.05344105, -0.02284033,  0.05260049, -0.01932774, -0.09711266,
       -0.19951601,  0.00738851, -0.07712144, -0.07025649, -0.03221126,
       -0.13667656,  0.08364721, -0.04125704, -0.03211425,  0.06983157,
       -0.00799658, -0.01177289, -0.16706869,  0.07463736, -0.09091619,
       -0.11201481,  0.08337475,  0.05930549, -0.07940312, -0.05631904,
       -0.01991395, -0.02349721,  0.1824768 ,  0.01132163, -0.04610476,
        0.01006832,  0.04507252, -0.08544502,  0.02825145,  0.02216966,
        0.03565323, -0.06984831,  0.03623525,  0.1129318 , -0.09319748,
       -0.04915803,  0.11094059,  0.0029751 , -0.08399934,  0.21738739,
        0.06743176, -0.09108514,  0.01798326, -0.08503798,  0.0290214 ,
        0.07596923, -0.02369601,  0.11986063, -0.04430954, -0.07